In [0]:
# ---------------------------------------------------------------------------- #
#                                Cell 1                                        #
# ---------------------------------------------------------------------------- #

# Before starting this project it is necessary to activate the kitty and corgi
# mode and set the power level to full power. This is necessary to gain 110% 
# programming power. You can do this in the settings, which open in the upper
# right corner. The settings are under miscellaneous.

# After you have done this, we need to add GPU support, which you can do by
# clicking on [runtime] in the options above this code snippet. Then you can
# select [change runtime type] and select [GPU] for hardware acceleration.
# Congrats, now you have a powerful remote machine up and running.

# Now we do some console commands to install some necessary libraries, we want
# to use for the project. The key to do console commands is the [!] symbol. We
# need this, because we are currently in a python console and want to run shell
# commands.
! pip install numpy
! pip install torch
! apt-get install python-opengl -y
! apt install xvfb -y
! pip install gym
! pip install gym[atari]
! pip install pyvirtualdisplay
! pip install piglet
! pip install gym[box2d]
! apt-get install x11-utils
! pip install tqdm
# Sometimes all these packages are already installed. But we want to make sure
# that this is definetely the case. After we have installed all the necessary
# python packages, we can start programming some python code.

In [0]:
# If you don't want to see the output of our console commands your can put 
# [ > /dev/null 2>&1] behind your command to mute the output
# (send it to nirvana)
# See this example:
! echo "Hello world!"
! echo "Hello world!"  > /dev/null 2>&1
! echo "Where is the second ,Hello World'?"

In [0]:
# ---------------------------------------------------------------------------- #
#                                Cell 2                                        #
# ---------------------------------------------------------------------------- #
# In this field we want to test, if we can import all the packages we want to
# use. This is not necessary as an extra cell but it is easier to detect if we
# made any installation errors.
import torch
import numpy as np
import gym
import time
import glob
import io
import base64
from tqdm import tqdm
# If it is possible to run this block, all your packages are installed.

# The game can only start, if there are some programs running on the machine
# that can render images. Since this remote machine normally doesn't have a
# display attached, we needed to install these in cell 1. But to run them, we 
# need to simulate a display, so our machine runs these programmes. We will do
# this with the following python commands.
import matplotlib.pyplot as plt
%matplotlib inline
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
from IPython import display as ipythondisplay
from IPython.display import HTML

In [0]:
# ---------------------------------------------------------------------------- #
#                                Cell 3                                        #
# ---------------------------------------------------------------------------- #
# After the simulation for a display is now running, we need to tell the system
# that it now has a monitor. We will do this from python with the following
# lines.
import os
if type(os.environ.get("DISPLAY")) is not str\
 or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [0]:
# Now we will set up the car racing simulator. This is just an example!
# You are not required to run this cell.
# At this point we just plot the game, make some random actions and refresh the
# plot. This makes it very slow, but we can see that it does something.
env = gym.make('CarRacing-v0')
env.reset()

fig = plt.figure()
ax = fig.add_subplot(111)
fig.show()

for _ in range(20):
  plt.imshow(env.render(mode='rgb_array'))
  ipythondisplay.display(plt.gcf())    
  ipythondisplay.clear_output(wait=True)
  env.step(env.action_space.sample())
env.close()

In [0]:
# ---------------------------------------------------------------------------- #
#                                Cell 4                                        #
# ---------------------------------------------------------------------------- #
# This cell defines some functions that we will use to record the rendered
# images as a video. There is also a function that can display the video. This
# is by far quicker than displaying the image over and over again. In this cell
# we will not use the functions. We will just use the cell to introduce the
# functions to the notebook, so we can use them later.

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  print(f'We have te following list of videos: {mp4list}')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = gym.wrappers.Monitor(env, './video', force=True)
  return env

In [0]:
# ---------------------------------------------------------------------------- #
#                                Cell 5                                        #
# ---------------------------------------------------------------------------- #

# no we will write a simple function, which always drives the car forward. This
# will be the part, where your solution comes in. The function will use the
# picture as an input and will output a steering signal. If you want to use your
# own code you can simply substitute the code in the function body. You can also
# import some of your already written code here.

def driver(img, step, model, container):

  # we will crop the image so the lower part and the upper part are not part of
  # the field of view of our driver. The lower pixels mainly contain some status
  # informations that are not necessary to detect a street. On the other hand, 
  # ysour vision model could maybe use this information.
  img = img[10:96, : , :]

  # find the pixels where there is a form of grey, which is not black [0, 0, 0]
  # or white [255, 255, 255]. These have the highest possibility of being street
  street_array = np.logical_and(np.logical_and(np.logical_and(
      img[:, :, 0] == img[:, :, 1], img[:, :, 1] == img[:, :, 2]), 
      img[:, :, 0] != 0), img[:, :, 0] != 255)

  # sepereate the image into three stripes. The dimensions of this array are
  # equal to height x width x rgb-channels. And then we count how many grey
  # pixels are in these parts
  left_part = np.sum(street_array[:, :32])
  middle_part = np.sum(street_array[:, 32:64])
  right_part = np.sum(street_array[:, 64:])

  # now we will decide for a steering signal. The signal has the following
  # shape: [Left=negative/right=positive, accelerate, break]
  if max([left_part, middle_part, right_part]) == left_part:
    steering_signal = [-0.5, 0, 0.2]
  elif max([left_part, middle_part, right_part]) == right_part:
    steering_signal = [0.5, 0, 0.2]
  elif max([left_part, middle_part, right_part]) == middle_part:
    steering_signal = [0, 0.5, 0]

  # we need the steering signal as a numpy array
  steering_signal = np.array(steering_signal)

  # append the last steering model to the container
  container.append(steering_signal)

  return steering_signal, container

# this function can also be alternated by you. It is used to load you model,
# since you dont want to load it over and over again.
def load_model():

  # initialize the model
  model = None

  # initialize the container
  container = []

  return model, container

In [0]:
# ---------------------------------------------------------------------------- #
#                                Cell 6                                        #
# ---------------------------------------------------------------------------- #

# Now we will set up a game environment, where we can play the game and get a
# video and the score of our solution at the end. This will be the script you
# should not alternate because we will use this one to evaluate your script. To 
# be able to run this, you will need to have your display activated. This cell
# requires the cells 1-5 to be already run before using it. We will probably
# change this part in the course of this semester, to have a fair evaluation,
# but for now you can use it as an example code.

# this is the line, where we introduce the environment and also force it to be
# recorded. The track is initialized randomly every time you run this cell.
env = wrap_env(gym.make('CarRacing-v0'))
observation = env.reset()

# make variable for the cumulative reward
c_reward = 0

# load the model and initialize the container
model, container = load_model()

# now we will iterate over multiple steps. The iteration will either end when we
# finished the track or end after we processed 1000 frames.
for i in range(300):
  # we need this to record a video. If you comment this, it won't be able to
  # record
  env.render()

  # here we use our steering function that we already introduced to the notebook
  # The inputs to this function is an observation, i.e. the current frame of the
  # game, the model and a variable named container. The model variable might be 
  # used  to process the observation. We introduced the container variable to
  # provide the possibility to save some information. In our example code the
  # container is used to save a history of steering commands.
  steering, container = driver(observation, i, model, container)

  # here we evaluate your choice and generate the next observation
  observation, reward, done, _ = env.step(steering)

  # accumulate the reward
  c_reward += reward

  # check whether the route is done
  if done:
    break

env.close()
print(f'We received a reward of: {c_reward} for this track.')

show_video()

In [0]:
# This Cell is just to show you, how we can use the container to look up all the
# steering signals that we did.
print(len(container))
print(container)